# Testing the non-Markovian Path Analisys Package

We are going to generate first a MC trajectory of a toy model 

In [1]:
import numpy as np

from NMpathAnalysis.nmtools.interval import Interval
from src.ensemble import *

In [2]:
'''
Function to mimic a MC simulation
'''
def mc_simulation(numsteps):
    x = 5
    I = Interval([0,100])
    mc_traj = []
    
    for i in range(numsteps):
        dx = np.random.uniform(-10,10)
        if (x + dx) in I:
            x = x + dx
        mc_traj.append(x)
    return np.array(mc_traj)

In [3]:
'''
Simple mapping funcion 
'''
def mapping_function(x):
    return int(x/10)

In [4]:
#Generating a short MC trajectoy
mc_trajectory = mc_simulation(10000)


## 1- Ensemble class (analysis of continuos trajectories)

Stores an esemble (list) of trajectories (array) with extra functionalities. The ensemble could have a any number of trajectories including no trajectories at all.

### Transition probabilities

In [5]:
my_ensemble = Ensemble(mc_trajectory)

In [6]:
n_states = 10
C1 = my_ensemble._count_matrix(n_states, mapping_function)
print(C1)

[[ 1016.   313.     0.     0.     0.     0.     0.     0.     0.     0.]
 [  312.   510.   284.     0.     0.     0.     0.     0.     0.     0.]
 [    0.   283.   596.   261.     0.     0.     0.     0.     0.     0.]
 [    0.     0.   260.   449.   239.     0.     0.     0.     0.     0.]
 [    0.     0.     0.   238.   482.   224.     0.     0.     0.     0.]
 [    0.     0.     0.     0.   223.   469.   254.     0.     0.     0.]
 [    0.     0.     0.     0.     0.   254.   480.   250.     0.     0.]
 [    0.     0.     0.     0.     0.     0.   250.   473.   220.     0.]
 [    0.     0.     0.     0.     0.     0.     0.   220.   407.   208.]
 [    0.     0.     0.     0.     0.     0.     0.     0.   208.   616.]]


In [7]:
K1 = my_ensemble._mle_transition_matrix(n_states, mapping_function)
print(K1)

[[ 0.76448457  0.23551543  0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.28209765  0.46112116  0.25678119  0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.24824561  0.52280702  0.22894737  0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.2742616   0.47362869  0.2521097   0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.25211864  0.51059322  0.23728814
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.23572939  0.49577167
   0.26849894  0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.25813008
   0.48780488  0.25406504  0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.26511135  0.50159067  0.23329799  0.        ]
 [ 0.          0.          0.          0

### Defining states and computing MFPTs
The states are considered intervals in the is the class is Ensemble

In [8]:
stateA = [0,10]
stateB = [90,100]

my_ensemble.mfpts(stateA, stateB)

{'mfptAB': 309.94444444444446,
 'mfptBA': 221.38888888888889,
 'std_err_mfptAB': 46.121717135154448,
 'std_err_mfptBA': 22.117829644949747}

### Adding more trajectories to the ensemble or (ensemble + ensemble)

In [9]:
seq1 = mc_simulation(5000)
seq2 = mc_simulation(5000)
seq3 = mc_simulation(5000)

my_e1 = Ensemble(seq1)
my_e2 = Ensemble(seq2)
my_e3 = Ensemble(seq3)

ensemble1 = my_e1 + my_e2 + my_e3

In [10]:
stateA = [0,10]
stateB = [90,100]
ensemble1.mfpts(stateA,stateB)

{'mfptAB': 239.85714285714286,
 'mfptBA': 279.10714285714283,
 'std_err_mfptAB': 34.784860442942488,
 'std_err_mfptBA': 38.30104767861949}

In [11]:
e1 = Ensemble([1,2,3,4])
e2 = Ensemble([2,3,4,5])
e3 = Ensemble([2,1,1,4])

my_ensembles = [e1, e2, e3]

ensemble_tot = Ensemble([])

for traj in my_ensembles:
    ensemble_tot += traj

ensemble_tot.trajectories
ensemble_tot.mfpts([1,1],[4,4])

{'mfptAB': 2.5,
 'mfptBA': 'NaN',
 'std_err_mfptAB': 0.35355339059327373,
 'std_err_mfptBA': 'NaN'}


## 2- DiscreteEnsemble class

We can generate a discrete trajectory from the same mapping function and we should obtain exaclty the same result:

In [12]:

d_ens = DiscreteEnsemble.from_ensemble(mc_trajectory,mapping_function)

print(d_ens.trajectories[0][1:200])
np.array(d_ens.trajectories).shape

[0 1 1 0 0 0 0 0 1 0 1 2 2 2 2 2 3 4 3 2 2 3 2 3 2 2 2 2 1 1 1 1 2 2 3 2 2
 2 3 4 4 5 4 5 4 4 5 5 5 5 5 4 5 5 4 5 6 6 6 7 7 8 7 8 8 8 7 7 7 6 6 6 7 6
 7 8 7 6 6 5 6 6 6 6 5 6 5 6 6 6 6 6 7 6 5 5 4 4 4 5 4 5 5 4 4 3 4 4 5 5 5
 5 5 4 3 3 3 3 2 3 2 2 2 1 2 1 2 2 2 2 3 3 3 3 3 3 2 2 3 3 3 3 4 4 3 3 4 4
 3 3 3 2 3 2 1 2 1 2 1 1 1 1 1 1 2 3 2 2 2 1 2 1 1 1 2 1 2 2 1 2 2 2 2 2 1
 1 0 0 0 1 0 0 0 0 1 2 1 0 0]


(1, 10000)

In [13]:
C2 = d_ens._count_matrix(n_states)
print(C2)

[[ 1016.   313.     0.     0.     0.     0.     0.     0.     0.     0.]
 [  312.   510.   284.     0.     0.     0.     0.     0.     0.     0.]
 [    0.   283.   596.   261.     0.     0.     0.     0.     0.     0.]
 [    0.     0.   260.   449.   239.     0.     0.     0.     0.     0.]
 [    0.     0.     0.   238.   482.   224.     0.     0.     0.     0.]
 [    0.     0.     0.     0.   223.   469.   254.     0.     0.     0.]
 [    0.     0.     0.     0.     0.   254.   480.   250.     0.     0.]
 [    0.     0.     0.     0.     0.     0.   250.   473.   220.     0.]
 [    0.     0.     0.     0.     0.     0.     0.   220.   407.   208.]
 [    0.     0.     0.     0.     0.     0.     0.     0.   208.   616.]]


In [14]:
K2= d_ens._mle_transition_matrix(n_states)
print(K2)

[[ 0.76448457  0.23551543  0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.28209765  0.46112116  0.25678119  0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.24824561  0.52280702  0.22894737  0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.2742616   0.47362869  0.2521097   0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.25211864  0.51059322  0.23728814
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.23572939  0.49577167
   0.26849894  0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.25813008
   0.48780488  0.25406504  0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.26511135  0.50159067  0.23329799  0.        ]
 [ 0.          0.          0.          0

### Defining states and computing MFPTs
The states are now considered sets, defining the states as follow we should obtain the same results

In [15]:
stateA = [0]
stateB = [9]

d_ens.mfpts(stateA, stateB)

{'mfptAB': 309.94444444444446,
 'mfptBA': 221.38888888888889,
 'std_err_mfptAB': 46.121717135154448,
 'std_err_mfptBA': 22.117829644949747}

The main function of this class would be to generate trajectories from the transition matrix

In [16]:
d_ens2 = DiscreteEnsemble.from_transition_matrix(K2, sim_length = 10000)

In [17]:
d_ens2.mfpts(stateA,stateB)

{'mfptAB': 182.90625,
 'mfptBA': 133.2258064516129,
 'std_err_mfptAB': 25.483157710619317,
 'std_err_mfptBA': 18.420948446819274}

In [25]:
pathEnsemble = DiscretePathEnsemble.from_transition_matrix(K2,stateA, stateB, n_paths = 10,ini_pops = [1])